In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import Imputer

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Let's name our data frames *movies* and *credits*

In [4]:
movies = pd.read_csv('../input/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb_5000_credits.csv')


Let's see what our data frames look like:

In [5]:
movies.head()

In [6]:
credits.head()

In [7]:
(credits['title']==movies['title']).describe()

Apparently, all th rows of both data frames are about the same movie, so when we want to merge the data frames, we can just easily attach all the columns from the credits data frame to the movies data frame. Before we do this, we remove the titles column from credits

In [8]:
del credits['title']
df = pd.concat([movies, credits], axis=1)

Let's take a look at our new data frame

In [9]:
df.head()

The concatenation worked. However, the columns are a bit in an awkward order and columns like homepage aren't that interesting for us. We choose the interesting columns, put them in a nice order and create a new data frame

In [10]:
newCols = ['id','title','release_date','popularity','vote_average','vote_count',
           'budget','revenue','genres','keywords','cast','crew','tagline', 'runtime', 'production_companies', 
           'production_countries', 'status']

df2 = df[newCols]

Let's explore our data frame a bit more in depth, let's take a look at our numerical columns

In [11]:
df2.describe().round()

Note that runtime consists of a few empty values, before we can really work with our data frame, we need to solve this. We use an imputer for this:

In [12]:
my_imputer = Imputer()

temp=df2
X2 = my_imputer.fit_transform(df2[['runtime']])
df2['runtime'] = X2


Change genre stuff, I won't go too far into this, because of the dictionary stuff

In [13]:
temp = df2['genres']
df2['genres']
for j in range(len(df2['genres'])):
    if "Science Fiction" in df2['genres'][j]:
        df2['genres'][j]=df2['genres'][j].replace("Science Fiction", "Sciencefiction")

for j in range(len(df2['genres'])):
    if "TV Movie" in df2['genres'][j]:
        df2['genres'][j]=df2['genres'][j].replace("TV Movie", "TVMovie")

Split the strings to the genre ID's:

In [14]:
genreArray = []*len(df2['genres'])
for j in range(len(df2['genres'])):
    string = df2['genres'][j]
    newArray = []*10
    for i in [1,5,9,13]:
         if i < len(string.split()):
              newArray.append(string.split()[i][:-1])
    genreArray.append(newArray)
    
df2['num_genres']=genreArray
test = map(int, df2['num_genres'])

Finish the Db so they become integers

In [15]:
temp = df2['num_genres']
test = []*len(temp)
for i in range(len(temp)):
    test.append(list(map(int, temp[i])))
    
df2['genreArray_Int'] = test

In [16]:
#A function to find certain values in a list in lists
def find_in_sublists(lst, value):
    for sub_i, sublist in enumerate(lst):
        try:
            return (sub_i, sublist.index(value))
        except ValueError:
            pass

    raise ValueError('%s is not in lists' % value)

In [17]:
df2.head()

In [18]:
import json

In [19]:
json_columns = ['genres', 'keywords','production_countries','production_companies']

for column in json_columns:
    if df2[column][4800] == list:
        continue
    df2[column] = df2[column].apply(json.loads)

In [20]:
map(apply(json.loads), df2['genres'].

We want to change the genre array to a list of lists of factors in stead of integers

In [ ]:
X = df2['num_genres']

In [ ]:
X = df2['num_genres']
X2 = []*len(X)
for i in range(len(X)):
    X2.append(list(map(pd.Categorical, X[i])))

In [ ]:
df2['num_genres'].head()

In [ ]:
(X2[1])

The missing values of the column runtime are now estimated. Before we want to start making predictions, we actally want ot have a training and a test dataset, so let's split it up! We'll use 30% of our data for testing and 70% for training.

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df2, test_size=0.3)

In [22]:
test.head()
#train.head()

In [33]:
s = df2['genreArray_Int'].head()
s

In [44]:
s_enc = pd.factorize(s[0])

In [52]:
headers = ['id','g1','g2','g3','g4']
test = pd.DataFrame(dict(zip(headers, foo)))

In [59]:
test = pd.DataFrame(s[0])
test['1'] = pd.DataFrame(s[1])
test[2]= pd.DataFrame(s[2])
test

In [97]:
f = lambda x, n: x[n] if n < len(x) else None
n, m = max(map(len, s)), len(s)

pd.DataFrame(
    [[f(j,i) for j in s] for i in range(n)],
    columns=[range(len(s))])

In [93]:
test =pd.DataFrame(s[1])
test[1] = pd.DataFrame(s[21])
test

In [89]:
s

In [ ]:
''' 
from sklearn import linear_model 
x_train = train[['budget','revenue','runtime']]
y_train = train[['vote_average']]

x_test = test[['budget','revenue','runtime']]


#Create linear regression object
linear = linear_model.LinearRegression()


#Train the model using the training sets and check score
linear.fit(x_train, y_train)
linear.score(x_train, y_train)

#Equation coefficient and Intercept
print('Coefficient: \n', linear.coef_)
print('Intercept: \n', linear.intercept_)

#Predict Output
predicted = linear.predict(x_test)
y_test = test[['vote_average']] 

In [ ]:
'''
#plt.scatter(y_test,x_test['budget'])

import matplotlib.pyplot as plt

x = range(10)
y = range(300000000)
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(y_test,x_test['budget'], s=10, c='b', marker = "s", label="Real averages")
ax1.scatter(predicted, x_test['budget'], s=10, c='r', marker="o", label="predicted averages")